In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
from sklearn.pipeline import Pipeline
from pathlib import Path


from os.path import join, isfile
from os import path, scandir, listdir


import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Embedding,  Flatten
from tensorflow.keras.models import Model, Sequential
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop
import keras_tuner as kt

from tensorflow.data import Dataset
from sklearn.preprocessing import QuantileTransformer,  KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from sklearn import metrics
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

import gc
import warnings
warnings.simplefilter('ignore')

# Parameters

In [ ]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 2017
    CVSEED = 2017
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    BINS = 128
    #N_ITERS = 2
else:
    N_SPLITS = 5
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 300
    VERBOSE = 1000
    SEED = 2017
    CVSEED = 2017
    BINS = 128
    #N_ITERS = 10

In [ ]:
def set_seed(seed=2017):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
    
set_seed(SEED)

# Load Dataset

In [ ]:
INPUT = Path("../input/tabular-playground-series-oct-2021")

train = pd.DataFrame(pd.read_csv(INPUT / "train.csv")[target])
#test = pd.read_csv(INPUT / "test.csv")
test = pd.DataFrame()
sub = pd.read_csv(INPUT / "sample_submission.csv")

In [ ]:
def list_all_files(location='../input/tps-oct-lv0', pattern=None, recursive=True):
    """
    This function returns a list of files at a given location (including subfolders)
    
    - location: path to the directory to be searched
    - pattern: part of the file name to be searched (ex. pattern='.csv' would return all the csv files)
    - recursive: boolean, if True the function calls itself for every subdirectory it finds
    """
    subdirectories= [f.path for f in scandir(location) if f.is_dir()]
    files = [join(location, f) for f in listdir(location) if isfile(join(location, f))]
    if recursive:
        for directory in subdirectories:
            files.extend(list_all_files(directory))
    if pattern:
        files = [f for f in files if pattern in f]
    return files

In [ ]:
names = ['bizen', 'henke', 'hamza', '28smiles','kashif', 'kosta', 'kaveh', 'dlaststark', 'pca', 'xgb2']

In [ ]:
namesec = ['lonnie', 'hgb', 'xgb_d2s', 'ctb_d2s']

In [ ]:
pred = list_all_files(pattern='oof')


for i in range(len(names)):
    avv = []
    
    for file in pred:
        if names[i] in file.split('/')[3]:
            avv.append(np.load(file))
    train[names[i]] = np.mean(avv, axis=0)
            
pred = list_all_files(location='../input/tps-oct-lv0-sec', pattern='oof')

for i in range(len(namesec)):
    avv = []
    
    for file in pred:
        if namesec[i] in file.split('/')[3]:
            avv.append(np.load(file))
    train[namesec[i]] = np.mean(avv, axis=0)
            
            
    
train.columns

In [ ]:
pred = list_all_files(pattern='pred')


for i in range(len(names)):
    avv = []
    
    for file in pred:
        if names[i] in file.split('/')[3]:
            avv.append(np.load(file))
    test[names[i]] = np.mean(avv, axis=0)
            
pred = list_all_files(location='../input/tps-oct-lv0-sec', pattern='pred')

for i in range(len(namesec)):
    avv = []
    
    for file in pred:
        if namesec[i] in file.split('/')[3]:
            avv.append(np.load(file))
    test[namesec[i]] = np.mean(avv, axis=0)
    
test.columns

In [ ]:
avv = []
for i in range(5):
    avv.append(np.load("../input/tps-oct-lv0/"+str(i+2017)+"lgb_oof.npy"))
train['lgb'] = np.mean(avv, axis=0)

avv = []
for i in range(5):
    avv.append(np.load("../input/tps-oct-lv0/agg"+str(i+1)+"_xgb_oof.npy"))
train['xgb'] = np.mean(avv, axis=0)

avv = []
for i in range(5):
    avv.append(np.load("../input/tps-oct-lv0/"+str(i+2017)+"lgb_pred.npy"))
test['lgb'] = np.mean(avv, axis=0)

avv = []
for i in range(5):
    avv.append(np.load("../input/tps-oct-lv0/agg"+str(i+1)+"_xgb_pred.npy"))
test['xgb'] = np.mean(avv, axis=0)

# Preprocessing

In [ ]:
features = train.columns[1:]

pipe = Pipeline([
       # ('imputer', SimpleImputer(strategy='median',missing_values=np.nan)),
        ("scaler", QuantileTransformer(n_quantiles=BINS,output_distribution='normal')),
        ('bin', KBinsDiscretizer(n_bins=BINS, encode='ordinal',strategy='uniform'))
        ])
#train[features] = pipe.fit_transform(train[features])
#test[features] = pipe.transform(test[features])

In [ ]:
test[features]

In [ ]:
train[target]

# Model

In [ ]:
def make_model():
    
    lr = 0.0157
    dropout =0.125
    embed_dim =4
    hidden_dim = 144
    n_layers = 3
    act = 'swish'
    #dstep = hp.Int('decay_steps', min_value=2000, max_value=4000, step=200)
    drate = 0.885
    eps =3.6e-08
        
    
    inputs = Input(train[features].shape[1:])
    X = Embedding(input_dim=BINS, output_dim=embed_dim, embeddings_initializer = "glorot_normal")(inputs)
    X = Dropout(dropout)(X)
    #X = BatchNormalization()(X)
    X = Flatten()(X)
    
    for i in range(n_layers):
        #units = hp.Int('units_{i}'.format(i=i), min_value=8, max_value=256, step=8)
        X = layers.Dense(hidden_dim/(2**i), activation=act, kernel_initializer=tf.keras.initializers.GlorotNormal())(X)
        X = Dropout(dropout)(X)
        #X = BatchNormalization()(X)
    outputs = layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.GlorotNormal())(X)
    model = keras.Model(inputs, outputs)
    
    #learning_rate = hp.Float('learning_rate', min_value=3e-4, max_value=3e-3)
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=lr,
        decay_steps = 400,
        decay_rate= drate)

    optimizer = keras.optimizers.Adam(learning_rate=lr_schedule, epsilon=eps)
    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer=optimizer,
                  metrics=[tf.keras.metrics.AUC(name='aucroc')])
    #model.summary()
    return model

# NN

In [ ]:
def prediction(x, y, batch_size=1024, epochs=100):
    cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
    val_losses = np.zeros(x.shape[0])
    nn_pred = np.zeros(test.shape[0])
    
    for fold, (train_indices, val_indices) in enumerate(cv.split(x, y)):
      print(f"===== fold {fold} =====")
      x_train, x_valid = x.iloc[train_indices], x.iloc[val_indices]
      y_train, y_valid = y.iloc[train_indices], y.iloc[val_indices]
      x_test = test[features]
    
      gc.collect()
        
      x_train[features] = pipe.fit_transform(x_train[features])
      x_valid[features] = pipe.transform(x_valid[features])
      x_test[features] = pipe.transform(x_test[features])
        
      
    
      model = make_model()
      model.fit( x_train[features], y_train, 
                validation_data=(x_valid[features] , y_valid),
                shuffle=True,
                verbose=0,
                #callbacks=[model_checkpoint_callback],
                callbacks=[
                #tf.keras.callbacks.ReduceLROnPlateau(monitor='val_aucroc', mode='max', patience=2),
                tf.keras.callbacks.EarlyStopping(monitor='val_aucroc', mode='max', patience=5)  ],
                batch_size=batch_size, 
                epochs=epochs)
      val_losses[val_indices] += model.predict(x_valid[features] )[:,-1]
      nn_pred += model.predict(x_test[features] )[:,-1]

      auc = roc_auc_score(y_valid, val_losses[val_indices])
      print(f"fold {fold} - nn auc: {auc:.6f}")   
    
      del model
      gc.collect()
     
    nn_pred /= N_SPLITS
    print(f"oof nn_auc = {roc_auc_score(y, val_losses)}")
    
    np.save("nn_oof.npy", val_losses)
    np.save("nn_pred.npy", nn_pred)

    gc.collect()
    return nn_pred

In [ ]:
nn_pred = prediction(x=train[features], y=train[target], 
             batch_size=1024, 
             epochs=100,
             #validation_data=(x_val, y_val),
            )

In [ ]:
sub[target]=nn_pred
sub.to_csv('submission.csv', index=False)

sub

# Log

/////// all average /////////

/// 128 quant normal kbins uniform with dropout, dstep=400, decreasing hidden units ///
2017 ver1
